In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━ 173.5/363.4 MB 3.1 MB/s eta 0:01:01

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Load video
cap = cv2.VideoCapture("/content/drive/MyDrive/r.mp4")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define Video Writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Middle detection lane
MIDDLE_X = 500
LANE_WIDTH = 20

# Tracking Data
bottle_tracker = defaultdict(lambda: [])
crossed_bottles = set()
total_bottles = 0
defective_bottles = 0

# Define Yellow and Blue Color Ranges
lower_yellow, upper_yellow = np.array([20, 100, 100]), np.array([30, 255, 255])
lower_blue, upper_blue = np.array([90, 50, 50]), np.array([130, 255, 255])

# Function to check defects
def is_defective(frame, x1, y1, x2, y2):
    bottle_roi = frame[y1:y2, x1:x2]
    hsv = cv2.cvtColor(bottle_roi, cv2.COLOR_BGR2HSV)

    # 1. **Check for Yellow Cap**
    cap_roi = bottle_roi[:bottle_roi.shape[0] // 5, :]
    yellow_mask = cv2.inRange(cv2.cvtColor(cap_roi, cv2.COLOR_BGR2HSV), lower_yellow, upper_yellow)
    if cv2.countNonZero(yellow_mask) / (cap_roi.shape[0] * cap_roi.shape[1]) * 100 < 5:
        return True

    # 2. **Check for Blue Bottle**
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    if cv2.countNonZero(blue_mask) / (bottle_roi.shape[0] * bottle_roi.shape[1]) * 100 < 30:
        return True

    # 3. **Check for Label**
    label_roi = bottle_roi[bottle_roi.shape[0] // 3: 2 * bottle_roi.shape[0] // 3, :]
    if cv2.countNonZero(cv2.Canny(cv2.cvtColor(label_roi, cv2.COLOR_BGR2GRAY), 50, 150)) / (label_roi.shape[0] * label_roi.shape[1]) * 100 < 10:
        return True

    return False

# Process Video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, tracker="bytetrack.yaml")

    for result in results:
        if result.boxes.id is None:
            continue

        track_ids = result.boxes.id.int().cpu().tolist()
        boxes = result.boxes.xyxy.cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2

            bottle_tracker[track_id].append(cx)
            if len(bottle_tracker[track_id]) > 30:
                bottle_tracker[track_id].pop(0)

            if MIDDLE_X - LANE_WIDTH < cx < MIDDLE_X + LANE_WIDTH:
                if track_id not in crossed_bottles:
                    crossed_bottles.add(track_id)
                    global total_bottles, defective_bottles
                    total_bottles += 1

                    if is_defective(frame, x1, y1, x2, y2):
                        defective_bottles += 1
                        cv2.putText(frame, "DEFECTIVE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    defect_percentage = (defective_bottles / total_bottles * 100) if total_bottles else 0
    good_percentage = 100 - defect_percentage if total_bottles else 0

    cv2.putText(frame, f"Total: {total_bottles}", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Defective: {defective_bottles} ({defect_percentage:.2f}%)", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Good: {good_percentage:.2f}%", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    out.write(frame)  # Save processed frame to output video

cap.release()
out.release()
cv2.destroyAllWindows()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 329MB/s]


Streaming output truncated to the last 5000 lines.
0: 384x640 1 bottle, 16.3ms
Speed: 2.8ms preprocess, 16.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 8.0ms
Speed: 3.6ms preprocess, 8.0ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 13.0ms
Speed: 3.2ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 11.1ms
Speed: 2.7ms preprocess, 11.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 10.1ms
Speed: 3.1ms preprocess, 10.1ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 17.0ms
Speed: 4.3ms preprocess, 17.0ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 12.0ms
Speed: 3.1ms preprocess, 12.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 16.0ms
Speed: 2.8ms preprocess, 16.0ms inference